In [29]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

In [30]:
## First challenge: With general framework before improvements

def black_scholes_price(S,K,T,r,sigma, call = True):
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2)*T) / (sigma * (T**0.5))
    d2 = (d1 - sigma*T**0.5)
    if call:
        return S * norm.cdf(d1) - K * np.exp(-r*T)*norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [32]:
def implied_volatility(market_price, S, K, T, r, call = True, tol =1e-6, max_iter = 1000):
    # Initial guesses
    sigma = 0.2
    for _ in range(max_iter):
        # Calculate the option price with the current sigma
        bs_price = black_scholes_price(S, K, T, r, sigma, call)
        # Derivative of Black-Scholes price with respect to sigma (vega)
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)  # PDF of d1

        # Update sigma using Newton-Raphson
        sigma_new = sigma - (bs_price - market_price) / vega
        if abs(sigma_new - sigma) < tol:
            return sigma_new
        sigma = sigma_new
    raise ValueError("Implied volatility did not converge")

In [33]:
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

# Global Parameters:
r = 0.05 # General assumption for Risk-free rate

def plot_vol_surface(vol_surface):
    strike_prices = []
    time_to_maturities = []
    volatilities = []

    for entry in vol_surface:
        T = entry['T']
        for strike, iv in zip(entry['strike'], entry['implied_vols']):
            if iv is not None:
                strike_prices.append(strike)
                time_to_maturities.append(T)
                volatilities.append(iv)

    # Filter any missing volatilities before plotting
    valid_data = [(K, T, iv) for K, T, iv in zip(strike_prices, time_to_maturities, volatilities) if iv is not None]
    
    if not valid_data: # Checking if valid_data is not empty
        print("No valid data available for plotting")
    X,Y,Z = np.array(valid_data).T

    # 3D plot Generation
    fig = plt.figure(figsize = (10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X,Y,Z, c=Z, cmap='viridis', marker='o')
    ax.set_title("Implied Volatility Surface")
    ax.set_xlabel("Strike prices")
    ax.set_ylabel("Time to Maturity (Years)")
    ax.set_zlabel("Implied Volatility")
    plt.show()

def main(ticker):
    S, market_data = fetch_market_data(ticker)
    vol_surface = calculate_vol_surface(market_data, S = S, r = r)
    plot_vol_surface(vol_surface)

In [58]:
def calculate_time_to_maturity(expiry):
    expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
    today = datetime.utcnow() # Use UTC for consistency
    return (expiry_date - today).days / 365.0

In [35]:
def fetch_market_data(ticker):
    try:
        # Fetch option data for a stock (e.g., AAPL)   
        stock = yf.Ticker(ticker)
        # Getting expiration dates
        expirations = stock.options
        S = stock.history(period="1d")["Close"].iloc[-1] # Current Stock prices
    except Exception as e:
        print(f"Error fetching market data for {ticker}: {e}")
        return None, None

    # Fetch option chain for multiple expiration
    data = {}
    for expiry in expirations[:3]: # Limit to 3 expirations for simplicity.
        chain = stock.option_chain(expiry)
        calls = chain.calls.dropna(subset = ["lastPrice"]) # Filter out NaN prices
        data[expiry] = {
            'strike' : calls['strike'].tolist(),
            'lastPrice' : calls['lastPrice'].tolist(),
            'expiration' : expiry
        }
    
    return S, data

In [36]:
def calculate_vol_surface(market_data, S, r):
    """
    Calculate implied Volatility for options in the market data
    """
    
    vol_surface = []
    for expiry, options in market_data.items():
        T = calculate_time_to_maturity(expiry)
        strikes = options['strike']
        market_prices = options['lastPrice']
        implied_vols = []
        for K, market_price in zip(strikes, market_prices):
            try:
                iv = implied_volatility(market_price, S, K, T, r, call = True)
                implied_vols.append(iv)
            except ValueError as e:
                print(f"Implied Volatility not computed for strike {K} at expiry {expiry}:{e}")
                implied_vols.append(None)
        vol_surface.append({
            'expiry': expiry,
            'T': T,
            'strike': strikes,
            'implied_vols': implied_vols
        })
    
    return vol_surface

#### Possible reasons why Newton method not converging.

1. Market prices not aligning with theoretical prices predicted by BS model.
2. Options that are deep in-the-money or out-of-the-money, where Vega (sensitivity to volatility) is low.
3. Poor initial guess?
4. Data quality issue?


In [61]:
ticker = "AAPL"
S, data = fetch_market_data(ticker)
stock = yf.Ticker(ticker)
expirations = stock.options
first_expiration = expirations[0]
data[first_expiration]["expiration"]

'2025-01-17'

In [80]:
## Trying first positivity

def is_valid_call_price(market_price, S, K, T, r):
    intrinsic_value = max(S-K*np.exp(-r*T), 0)
    upper_bound = S
    
    if intrinsic_value <= market_price <= upper_bound:
        return True
    
    return False

for expiry in expirations[:3]:
    S = S
    market_prices = data[expiry]["lastPrice"]
    K = data[expiry]["strike"]
    T = calculate_time_to_maturity(expiry)
    r = 0.05
    
    invalid_entries= [
        (mp, sp) for mp, sp in zip(market_prices, K) if not is_valid_call_price(mp, S, sp, T, r)
    ]
    
    if invalid_entries:
        print("Found invalid option prices:")
        for market_price, strike_price in invalid_entries:
            print(f"Market Price: {market_price}, Strike Price: {strike_price}")
    else:
        print("All option prices are valid.")

Found invalid option prices:
Market Price: 232.65, Strike Price: 5.0
Market Price: 184.23, Strike Price: 45.0
Market Price: 179.4, Strike Price: 50.0
Market Price: 169.29, Strike Price: 60.0
Market Price: 164.54, Strike Price: 65.0
Market Price: 159.45, Strike Price: 70.0
Market Price: 154.23, Strike Price: 75.0
Market Price: 149.48, Strike Price: 80.0
Market Price: 144.42, Strike Price: 85.0
Market Price: 134.85, Strike Price: 95.0
Market Price: 129.45, Strike Price: 100.0
Market Price: 124.72, Strike Price: 105.0
Market Price: 119.45, Strike Price: 110.0
Market Price: 114.23, Strike Price: 115.0
Market Price: 109.41, Strike Price: 120.0
Market Price: 104.5, Strike Price: 125.0
Market Price: 99.25, Strike Price: 130.0
Market Price: 94.58, Strike Price: 135.0
Market Price: 89.55, Strike Price: 140.0
Market Price: 84.19, Strike Price: 145.0
Market Price: 79.15, Strike Price: 150.0
Market Price: 74.21, Strike Price: 155.0
Market Price: 69.5, Strike Price: 160.0
Market Price: 63.9, Strike

C:\Users\ASUS\AppData\Local\Temp\ipykernel_6172\4178437435.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow() # Use UTC for consistency
